In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#-----------------------------------------------------------------------------------------------------------        
# テータ読み込み
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
submission_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")

# 乱数を固定する
import random
np.random.seed(1234)
random.seed(1234)

# データを可視化する
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")
#-----------------------------------------------------------------------------------------------------------        

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


#### 手順
#### 説明変数の理解
#### ベースライン
#### 説明変数を適切処理
#### 順次改善

In [2]:
# 全データ結合
all_df = pd.concat([train_df,test_df],sort=False).reset_index(drop=True)

#### 欠損値補完、カテゴリ変数処理

#### データの把握
#### 提出データ: id = 1459行, Id="1461-2919"のSalePriceを予測する

【目的変数】<br>
SalePrice: 販売価格<br>

【カテゴリ】<br>
MSZoning(NA=4): 用途向け(農家、住居、産業)、
Exterior1st(NA=1): 外設備、
Exterior2nd(NA=1): 外設備、
MasVnrType(NA=23): 石積みベニヤ、
BsmtQual(NA=81): 地下の深さ、BsmtCond(NA=82): 地下の状態、
BsmtExposure(NA=82): 散歩道と庭壁の露出、
BsmtFinType1(NA=79): 地下面積の評価、
BsmtFinType2(NA=80): 地下面積の評価、
Electrical(NA=1): 電気容量、
KitchenQual(NA=1): キッチンの質、
Functional(NA=2): 家の機能、
GarageType(NA=157): ガレージの場所、
GarageFinish(NA=159): ガレージのインテリア性、
GarageQual(NA=159): ガレージの質、
GarageCond(NA=159): ガレージ状態(択一)、
SaleType(NA=1): 支払い方法、
Street: 道路(舗装状態)、
LotShape: 家の形状、
LandContour: 平坦さ、
LotConfig: 区画、
LandSlope: スロープ、
Neighborhood: 地域(Ames市内)、
BldgType: 誰向け(単身、世帯)、
HouseStyle: 間取り、
RoofStyle: 屋根の形状、
RoofMatl: 屋根の材質、
ExterQual: 外設備の評価、
ExterCond: 外設備状態の評価、
Foundation: 基礎、
Heating: 暖房、HeatingQC: 暖房の質、
CentralAir: エアコン集中管理、
PaveDrive: 道路舗装、
SeleCondition: 売り出し状況、<br>

(まとめ)<br>
Condition = Condition1 + Condition2
Utilities(NA=2): すべてAppPubで置換、<br>


(NA多すぎてDrop)<br>
Alley(NA=2721): 路地(舗装状態)・・・Streetと同様、
FireplaceQu(NA=1420): 防火スペースの質、
MiscFeature(NA=2814): その他、<br>
PoolQC(NA=2909): プール質、
Fence(NA=2348): フェンス質、


【数値】<br>
(重要)<br>
OverallQual: 全体の評価(10段階)、<br>
GrLivArea: 良質面積、
TotalBsmtSF(NA=1): 合計面積、
1stFlrSF: 1階床面積、
BsmtFinSF1: 面積、
GarageCars(NA=1): ガレージのキャパシティ、<br>

(普通)<br>
MSSubClass: 誰向け(単身、世帯)、
LotArea: 築面積、
YearBuilt: 建造年、
YearRemodAdd: リフォーム年、
OverallCond: 状態の全体評価(10段階)、
MasVnrArea: ベニヤの面積、
BsmtFinSF2: 面積、BsmtUnfSF(NA=1): 未完成部分の面積、
2ndFlrSF: 2階床面積、
LowQualFinSF: 悪質面積、
BsmtFullBath(NA=2): 地下の浴槽、
BsmtHalfBath(NA=2): 地下の半浴槽、
FullBath: フル浴槽、
HalfBath: 半浴槽、
BedroobAbvGr: 良質寝室、
KitchenAbvGr: 良質キッチン、
TotRmsAbvGrd: 良質部屋数、
Fireplaces: 消化器の数、
GarageYrBlt(NA=159): ガレージの築年数、
GarageArea(NA=1): ガレージ面積、
WoodDeckSF: ベランダ面積、
OpenPorchSF: 掘り出し玄関面積、
EnclosedPorch: 囲み玄関、
3SsnPorch: 3季節玄関、
ScreenPorch: スクリーン玄関、
PoolArea: プール面積、
MiscVal: その他(＄)、
MoSold: 月家賃、
YrSold: 年家賃、<br>

(NA多すぎてDrop)<br>
LotFrontage(NA=486): 接する道への距離、


In [3]:
# # 売れた年月日など不要では？
# # all_df["MoSold"].hist()
# tmp = all_df[["Id","YrSold","YearRemodAdd"]].groupby(["YrSold","YearRemodAdd"]).count().unstack()
# plt.figure(figsize=(15,8))
# sns.heatmap(tmp,annot=True)
# # all_df[["MoSold","YrSold"]]

In [4]:
# # 変数をまとめる
# all_df['Condition'] = all_df['Condition1'] + all_df['Condition2']
# all_df.drop(columns=['Condition1','Condition2'],inplace=True)

# all_df['Exterior'] = all_df['Exterior1st'] + all_df['Exterior2nd']
# all_df.drop(columns=['Exterior1st','Exterior2nd'],inplace=True)

# # Utitlitiesは99%AllPubのため
# all_df['Utilities'].fillna('AllPub',inplace=True)
# # 欠損値を落とす
# # NA多数につき落とすカラム
# dcol = ['Alley', 'FireplaceQu', 'MiscFeature', 'LotFrontage', 'PoolQC', 'Fence']
# data = all_df.drop(columns = dcol, axis=1)

# # 初回はNAありでやってみる
# # data = data.dropna()

In [5]:
# ラベルを落とす
all_df.drop(columns="Id", inplace = True)

In [6]:
# カテゴリ変数をラベルエンコードする
from sklearn.preprocessing import LabelEncoder

categories = all_df.columns[all_df.dtypes == 'object']
print(categories)

for cat in categories:
    le = LabelEncoder()
#     print(cat)
    le = le.fit(all_df[cat])
    all_df[cat] = le.transform(all_df[cat])

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')


In [7]:
# 目的変数を対数変換する
all_df["SalePrice"] = np.log(all_df["SalePrice"])
train_X = all_df.loc[~all_df['SalePrice'].isnull(),:].drop(columns="SalePrice", axis=1).reset_index(drop=True)
train_Y = all_df.loc[~all_df['SalePrice'].isnull(),:]["SalePrice"].reset_index(drop=True)
test_X = all_df.loc[all_df['SalePrice'].isnull(),:].drop(columns="SalePrice", axis=1).reset_index(drop=True)
test_Y = submission_df

In [8]:
# データ分割
# train_X = all_df.loc[~all_df['SalePrice'].isnull(),:].drop(columns="SalePrice", axis=1).reset_index(drop=True)
# train_Y = all_df.loc[~all_df['SalePrice'].isnull(),:]["SalePrice"].reset_index(drop=True)
# test_X = all_df.loc[all_df['SalePrice'].isnull(),:].drop(columns="SalePrice", axis=1).reset_index(drop=True)
# test_Y = submission_df

In [9]:
# # 訓練データと検証データに分割
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(train_X, train_Y, test_size=0.2)

In [10]:
# モデル
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# データセット生成(訓練用、検証用)
lgb_train = lgb.Dataset(X_train, Y_train)
lgb_val = lgb.Dataset(X_val, Y_val, reference = lgb_train)

# パラメータ設定
lgbm_params = {
    "objective": "regression",#回帰
    "metric": "rmse",#評価関数
    "boosting": "gbdt",#ブースティング
    "num_iteration": 1000,#ブースティング繰り返し回数
    "learning_rate": 0.1,#学習率
    "verbosity": 1,#冗長性(状態表示)
    "seed": 1234,#乱数生成番号
    "early_stopping_rounds": 20,#評価関数値が連続して下がらないと打ち止め
}

# 上記パラメータでモデルを学習する
model = lgb.train(params = lgbm_params,
                 train_set = lgb_train,
                 valid_sets = lgb_val,)

y_pred = model.predict(X_val, num_iteration=model.best_iteration)
# rmse = np.sqrt(mean_squared_error(np.log(Y_val), np.log(y_pred)))
# すでに対数変換した場合
rmse = np.sqrt(mean_squared_error(Y_val, y_pred))
print('RMSE',round(rmse,3))

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005201 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3137
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 74
[LightGBM] [Info] Start training from score 12.027564
[1]	valid_0's rmse: 0.382149
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 0.354821
[3]	valid_0's rmse: 0.330437
[4]	valid_0's rmse: 0.310203
[5]	valid_0's rmse: 0.291736
[6]	valid_0's rmse: 0.275426
[7]	valid_0's rmse: 0.260845
[8]	valid_0's rmse: 0.247152
[9]	valid_0's rmse: 0.236827
[10]	valid_0's rmse: 0.227242
[11]	valid_0's rmse: 0.21793
[12]	valid_0's rmse: 0.210368
[13]	valid_0's rmse: 0.203214
[14]	valid_0's rmse: 0.196921
[15]	valid_0's rmse: 0.191173
[16]	valid_0's rmse: 0.18597
[17]	valid_0's rmse: 0.181887
[18]	valid_0's rmse: 0.17821
[1

In [11]:
# # モデル構築
# import lightgbm as lgb
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import KFold
# from sklearn.model_selection import train_test_split

# # X_train, X_val, Y_train, Y_val = train_test_split(train_X, train_Y, test_size=0.2)

# # パラメータ設定
# lgbm_params = {
#     "objective": "regression",#回帰
#     "metric": "rmse",#評価関数
#     "boosting": "gbdt",#ブースティング
#     "num_iteration": 1000,#ブースティング繰り返し回数
#     "learning_rate": 0.1,#学習率
#     "verbosity": 1,#冗長性(状態表示)
#     "seed": 1234,#固定乱数生成
#     "early_stopping_rounds": 20,#評価関数値が連続して下がらないと打ち止め
# }

# # データセット分割数
# folds = 5
# KF = KFold(n_splits=folds)

# # パラメータ一覧
# models = []
# rmses = []
# # out of fold; 目的変数の予測値
# oof = np.zeros(len(train_X))

# for train_index, val_index in KF.split(X_train):
#     X_train = train_X.loc[train_index]
#     X_val = train_X.loc[val_index]
#     Y_train = train_Y.loc[train_index]
#     Y_val = train_Y.loc[val_index]

#     # データセット生成(訓練用、検証用)
#     lgb_train = lgb.Dataset(X_train, Y_train)
#     lgb_val = lgb.Dataset(X_val, Y_val, reference = lgb_train)

#     # 上記パラメータでモデルを学習する
#     model = lgb.train(params = lgbm_params,
#                      train_set = lgb_train,
#                      valid_sets = lgb_val)

#     y_pred = model.predict(X_val, num_iteration=model.best_iteration)
# #     tmp_rmse = np.sqrt(mean_squared_error(np.log(Y_val), np.log(y_pred)))# 
#     # すでに対数変換した場合
#     tmp_rmse = np.sqrt(mean_squared_error(Y_val, y_pred))

#     print('RMSE',round(rmse,3))
#     models.append(model)
#     rmses.append(tmp_rmse)
#     oof[val_index] = y_pred

In [12]:
# sum(rmses)/len(rmses)

In [13]:
# # ベストスコアを出したモデルを抽出する
# i = 0
# j = 0
# score = 0
# best = 1e6
# for model in models:
#         score = model.best_score['valid_0']['rmse']
#         print(i,"回目: ",score)
#         if best > score:
#             best = score
#             j = i
#         i = i + 1
        
# print("ベストスコアは、",j,"回目の、",np.round(best,2),"に決定")

In [14]:
# submissionデータで予測する
y_pred = model.predict(test_X, num_iteration=model.best_iteration)
submission_df['SalePrice'] = np.exp(y_pred)

In [15]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)